In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from zipfile import ZipFile
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stopwords/stopwords.txt
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv
/kaggle/input/bad-bad-words/bad-words.csv
/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv
/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv
/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv
/kaggle/input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.twitter.27B.25d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.twitter.27B.50d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.twitter.27B.100d.txt


In [2]:
sample_data = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv')
sample_data.head()

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


In [3]:
test = sample_data['text']
test.head()

0    "\n \n\nGjalexei, you asked about whether ther...
1    Looks like be have an abuser , can you please ...
2    I confess to having complete (and apparently b...
3    "\n\nFreud's ideas are certainly much discusse...
4    It is not just you. This is a laundry list of ...
Name: text, dtype: object

In [4]:
lis = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
lis2 = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv')

In [5]:
lis.shape

(159571, 8)

In [6]:
lis['compound'] = lis.sum(axis = 1)
def a(x) :
    if x> 0:
        return 1
    else:
        return 0
lis['y'] = lis['compound'].apply(lambda x: a(x))
lis.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,compound,y
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,0


In [7]:
lis = lis[['comment_text', 'y']]
lis.head()

,comment_text,y
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
less_arr = pd.DataFrame(lis2['less_toxic']).rename(columns = {'less_toxic':'comment_text'}, inplace = False).drop_duplicates()
more_arr = pd.DataFrame(lis2['more_toxic']).rename(columns = {'more_toxic':'comment_text'}, inplace = False)
more_arr['y'] = 1
less_arr['y'] =  0  
lis = pd.concat([less_arr, more_arr, lis])
lis.tail()

,comment_text,y
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


In [9]:
lis.shape

(201211, 2)

In [10]:
contractions_dict = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions in the reviews
lis['comment_text']= lis['comment_text'].apply(lambda x:expand_contractions(x))

lis.head()

,comment_text,y
0,This article sucks \n\nwoo woo wooooooo,0
1,"""And yes, people should recognize that but the...",0
2,"Western Media?\n\nYup, because every crime in...",0
3,And you removed it! You numbskull! I do not ca...,0
4,smelly vagina \n\nBluerasberry why do not you...,0


In [11]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]

def clean(data, col):
    
    data[col] = data[col].str.replace('https?://\S+|www\.\S+', ' social medium ')      
        
    data[col] = data[col].str.lower()
    data[col] = data[col].str.replace("4", "a") 
    data[col] = data[col].str.replace("2", "l")
    data[col] = data[col].str.replace("5", "s") 
    data[col] = data[col].str.replace("1", "i") 
    data[col] = data[col].str.replace("!", "i") 
    data[col] = data[col].str.replace("|", "i") 
    data[col] = data[col].str.replace("0", "o") 
    data[col] = data[col].str.replace("l3", "b") 
    data[col] = data[col].str.replace("7", "t") 
    data[col] = data[col].str.replace("7", "+") 
    data[col] = data[col].str.replace("8", "ate") 
    data[col] = data[col].str.replace("3", "e") 
    data[col] = data[col].str.replace("9", "g")
    data[col] = data[col].str.replace("6", "g")
    data[col] = data[col].str.replace("@", "a")
    data[col] = data[col].str.replace("$", "s")
    data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")    
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace("centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", " sick ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
    data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
    data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ")
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
    data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ")
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you')
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " is ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace('\s+', ' ')  # will remove more than one whitespace character
#     text = re.sub(r'\b([^\W\d_]+)(\s+\1)+\b', r'\1', re.sub(r'\W+', ' ', text).strip(), flags=re.I)  # remove repeating words coming immediately one after another
    data[col] = data[col].str.replace(r'(.)\1+', r'\1\1') # 2 or more characters are replaced by 2 characters
#     text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
    data[col] = data[col].str.replace("[:|♣|'|§|♠|*|/|?|=|%|&|-|#|•|~|^|>|<|►|_]", '')
    
    
    data[col] = data[col].str.replace(r"what's", "what is ")    
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace(r"\'s", " ")

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    return data
a = clean(lis, 'comment_text')
test= test.apply(lambda x:expand_contractions(x))
test = clean(pd.DataFrame(test), 'text')
a.head()

,comment_text,y
0,article sucks woo woo woo,0
1,"""and yes, people recognize usually not. one fi...",0
2,"western media yup, every crime entire world ca...",0
3,"removed iti numbskulli care say anymore, lifei...",0
4,smelly vagina bluerasberry model cheesy blue v...,0


In [12]:
test.head()

,text
0,""" gjalexei, asked whether """"anti-editorializin..."
1,"looks like abuser , please look thanks."
2,confess complete (and apparently blissful) ign...
3,""" freud ideas certainly much discussed today, ..."
4,you. laundry list stupid allegations scooped g...


In [13]:
lis['comment_text'] = lis['comment_text'].str.lower()

In [14]:
test['text'] = test['text'].str.lower()

In [15]:
import string
lis['comment_text'] = lis['comment_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
lis.sample(5)

,comment_text,y
93204,mistake rude looks like responded wrong user ...,0
30020,bold textgreg thomas albuquerque new mexico gay,1
73835,right countless counted big number finite,0
134741,memorizing melodic sequences another neglected...,0
7459,i sorry wrote stupid sentence l clock night de...,0


In [16]:
test['text'] = test['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))

In [17]:
lis['comment_text'] = lis['comment_text'].apply(lambda x: re.sub('\W+',' ', x))
lis.head()

,comment_text,y
0,article sucks woo woo woo,0
1,and yes people recognize usually not one first...,0
2,western media yup every crime entire world cau...,0
3,removed iti numbskulli care say anymore lifei ...,0
4,smelly vagina bluerasberry model cheesy blue v...,0


In [18]:
test['text'] = test['text'].apply(lambda x: re.sub('\W+',' ', x))

In [19]:
lis = pd.concat([lis[lis['y'] > 0], lis[lis['y'] == 0].sample(int(len(lis[lis['y'] > 0])*1.5))], axis = 0).sample(frac = 1)

In [20]:
lis.shape

(115832, 2)

In [21]:
import unidecode
lis['comment_text'] = lis['comment_text'].apply(lambda x: unidecode.unidecode(x))

In [22]:
test['text'] = test['text'].apply(lambda x: unidecode.unidecode(x))

In [23]:
test.shape

(7537, 1)

In [24]:
submit, texts, outputs = [], [], []
for i in lis.itertuples():
    texts.append(i[1])
    outputs.append(i[-1])
for i in test.itertuples():
    submit.append(i[1])

In [25]:
stopwords = list(pd.read_table('/kaggle/input/stopwords/stopwords.txt'))
def sw(text):
    a = text.split(" ")
    for i in a:
        if i in stopwords == True:
            a.remove(i)
    return " ".join(a)
lis['comment_text'] = lis['comment_text'].apply(lambda x: sw(x))
test['text'] = test['text'].apply(lambda x: sw(x))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(lis['comment_text']), pd.DataFrame(lis['y']), test_size=0.30, random_state=101)

In [26]:
lis.head()

,comment_text,y
119394,redirect talkigie ais gill rail accident,0
57312,thanks thanks kind words thank well really tea...,0
6591,name yalma loser gets pissed people nothingii,1
9927,encourage idiot sectorooi say am keeps rewriti...,1
13851,thankyou blocking little shit block expired in...,1


In [27]:
import spacy
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, metrics, svm
from sklearn.utils import shuffle
nlp = spacy.load("en_core_web_lg")

In [28]:
class GloveVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 150

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([self.nlp(text).vector for text in X])

In [29]:
column_preprocessor = ColumnTransformer(
    [
        ('text_glove', GloveVectorTransformer(nlp), 'comment_text'),
    ],
    remainder='drop',
    n_jobs=1
)

In [30]:
pipeline = Pipeline([
    ('column_preprocessor', column_preprocessor),
    ('svm', svm.SVC(kernel='rbf', C=1.2, gamma=0.2))
])

In [31]:
X_train.shape

(81082, 1)

In [32]:
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_train)
print(metrics.accuracy_score(y_train, predictions))

0.8821810019486446


In [33]:
predictions = pipeline.predict(X_test)
print(metrics.accuracy_score(y_test, predictions))

0.8565755395683453


In [34]:
vec = TfidfVectorizer(analyzer='char_wb', max_df=0.7, min_df=1, ngram_range=(2, 5) )

In [35]:
X = vec.fit_transform(lis['comment_text'])

In [36]:
z= lis["y"].values
y=np.around ( z ,decimals = 2)
print(X)

  (0, 100198)	0.09013838842687467
  (0, 171074)	0.08344294899967009
  (0, 89554)	0.10225510630697336
  (0, 86085)	0.12474701392806065
  (0, 47938)	0.1245739559552314
  (0, 442)	0.12492174938718874
  (0, 123698)	0.04553159609406418
  (0, 100197)	0.0765628479682706
  (0, 171062)	0.07413271408435466
  (0, 89547)	0.08221369914891918
  (0, 86082)	0.12417650005645785
  (0, 47934)	0.12195990203176893
  (0, 437)	0.06197825451831063
  (0, 250267)	0.03622202105625668
  (0, 123697)	0.03719582245367059
  (0, 100079)	0.0638587584054455
  (0, 170966)	0.04913536089226228
  (0, 89534)	0.0809451759249195
  (0, 86074)	0.11677393391494775
  (0, 47909)	0.06085888708974739
  (0, 421)	0.05073856994111679
  (0, 250266)	0.028913948252956578
  (0, 122130)	0.02880467348444231
  (0, 99160)	0.03153250251277115
  (0, 170778)	0.039257311356041755
  :	:
  (115831, 202008)	0.04151438848850721
  (115831, 245138)	0.04985967996749077
  (115831, 61324)	0.04496595515493526
  (115831, 155966)	0.038825714431200815
  (115831

In [37]:
testx,valx,testy,valy = train_test_split(X_test, np.ravel(y_test), test_size=0.42, random_state=101)

In [ ]:
model.score(testx,testy)

In [ ]:
predictions = pipeline.predict(data_submit)

In [ ]:
sample_data['score'] = predictions
sample_data.head()
del sample_data['text']

In [ ]:
sample_data.to_csv('submission.csv', index = False, float_format='%.20f' )
